<a href="https://colab.research.google.com/github/Balghanimi/AutoGPT/blob/master/IraqSolarTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install the required libraries
!pip install reportlab openpyxl fuzzywuzzy pyTelegramBotAPI # Install pyTelegramBotAPI instead of python-telegram-bot
!pip install --upgrade pip  # It's good practice to keep pip updated

# The rest of your code follows
import requests
from bs4 import BeautifulSoup
import tkinter as tk
from tkinter import ttk
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from datetime import datetime
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import re
import os
import telebot  # For Telegram integration using pyTelegramBotAPI
from fuzzywuzzy import fuzz

# Install dependencies:
# pip install requests beautifulsoup4 reportlab pandas matplotlib openpyxl fuzzywuzzy pyTelegramBotAPI # Updated instruction
# Download DejaVuSans.ttf from https://sourceforge.net/projects/dejavu/files/dejavu/ and place in script directory
# For Telegram: Get TELEGRAM_TOKEN from BotFather and your chat_id

# Telegram configuration (replace with your token)
TELEGRAM_TOKEN = "your_bot_token_here"  # Replace with your Telegram bot token
CHAT_ID = None  # Will be set via GUI input

# Database setup
def setup_database():
    conn = sqlite3.connect('solar_offers.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS offers
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                 category TEXT,
                 title TEXT,
                 price REAL,
                 currency TEXT,
                 governorate TEXT,
                 source TEXT,
                 link TEXT,
                 date TEXT)''')
    conn.commit()
    conn.close()

# Categorize offers based on Arabic keywords
def categorize_offer(title, snippet=""):
    keywords = {
        "ألواح شمسية": ["لوح", "ألواح", "شمسي"],
        "محولات": ["محول", "عاكس", "تحويل"],
        "بطاريات": ["بطارية", "بطاريات", "تخزين"],
        "أنظمة كاملة": ["نظام", "أنظمة", "كامل"]
    }
    for category, kws in keywords.items():
        if any(kw in title or kw in snippet for kw in kws):
            return category
    return "أخرى"

# Google search for solar offers
def search_google():
    offers = []
    query = "عروض أنظمة الطاقة الشمسية أسعار مكونات العراق"
    url = f"https://www.google.com/search?q={query}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        results = soup.find_all('div', class_='tF2Cxc')[:5]

        for result in results:
            title = result.find('h3').text if result.find('h3') else "لا يوجد عنوان"
            snippet = result.find('span', class_='aCOpRe').text if result.find('span', class_='aCOpRe') else ""
            link = result.find('a')['href'] if result.find('a') else "لا يوجد رابط"

            offers.append({
                "title": title,
                "snippet": snippet,
                "price": 0,  # Google results often lack price data
                "currency": "Unknown",
                "governorate": "Unknown",  # Could be enhanced with snippet parsing
                "source": "Google",
                "link": link
            })
    except Exception as e:
        print(f"Error scraping Google: {e}")
        offers.append({
            "title": "خطأ", "snippet": f"فشل البحث: {e}", "price": 0,
            "currency": "Unknown", "governorate": "Unknown", "source": "Google", "link": "غير متاح"
        })
    return offers

# OpenSooq scraping
def scrape_opensooq():
    try:
        url = "https://iq.opensooq.com/ar/find?term=%D8%A7%D9%84%D8%B7%D8%A7%D9%82%D8%A9%20%D8%A7%D9%84%D8%B4%D9%85%D8%B3%D9%8A%D8%A9"
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        offers = []
        items = soup.find_all('li', class_='rectLi')[:10]

        for item in items:
            title = item.find('h2').text.strip() if item.find('h2') else "No title"
            price = item.find('span', class_='inline').text.strip() if item.find('span', class_='inline') else "0"
            location = item.find('span', class_='grey').text.strip() if item.find('span', class_='grey') else "Unknown"
            link = item.find('a')['href'] if item.find('a') else "#"

            try:
                price_num = float(re.sub(r'[^\d.]', '', price.split()[0])) if price != "0" else 0
                currency = "IQD" if "دينار" in price else "USD"
            except (ValueError, IndexError):
                price_num = 0
                currency = "Unknown"

            offers.append({
                "title": title,
                "snippet": "",  # OpenSooq doesn't provide snippets
                "price": price_num,
                "currency": currency,
                "governorate": location,
                "source": "OpenSooq",
                "link": f"https://iq.opensooq.com{link}"
            })
        return offers
    except Exception as e:
        print(f"Error scraping OpenSooq: {e}")
        return []

# Enhanced PDF generation with visualizations
def generate_pdf(offers):
    font_path = "DejaVuSans.ttf"
    if not os.path.exists(font_path):
        print("خطأ: ملف الخط DejaVuSans.ttf غير موجود. قم بتحميله ووضعه في مجلد المشروع.")
        return

    pdfmetrics.registerFont(TTFont("Arabic", font_path))
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"solar_offers_iraq_{timestamp}.pdf"

    doc = SimpleDocTemplate(filename, pagesize=letter)
    elements = []
    styles = getSampleStyleSheet()

    # Add title
    title = Paragraph("<para align=right><font name='Arabic' size=14>عروض أنظمة الطاقة الشمسية في العراق</font></para>", styles["Normal"])
    elements.append(title)

    # Create price visualization
    prices = [o['price'] for o in offers if o['price'] > 0]
    if prices:
        plt.figure(figsize=(6, 3))
        plt.hist(prices, bins=5, color='gold', edgecolor='black')
        plt.title('توزيع الأسعار')
        plt.xlabel('السعر (IQD)')
        plt.ylabel('العدد')
        imgdata = BytesIO()
        plt.savefig(imgdata, format='png')
        imgdata.seek(0)
        elements.append(Paragraph("<para align=center><font name='Arabic'>توزيع الأسعار</font></para>", styles["Normal"]))
        elements.append(Image(imgdata, width=400, height=200))
        plt.close()

    # Prepare data for table
    data = [["الفئة", "العنوان", "السعر", "المحافظة", "المصدر"]]
    for offer in offers:
        category = categorize_offer(offer['title'], offer.get('snippet', ''))
        data.append([
            category,
            offer.get('title', '')[:40],
            f"{offer.get('price', 0)} {offer.get('currency', '')}",
            offer.get('governorate', ''),
            offer.get('source', '')
        ])

    # Create table
    table = Table(data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'RIGHT'),
        ('FONTNAME', (0, 0), (-1, -1), 'Arabic'),
        ('FONTSIZE', (0, 0), (-1, -1), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    elements.append(table)

    doc.build(elements)
    print(f"تم إنشاء ملف PDF: {filename}")

# Send results to Telegram
def send_to_telegram(offers, chat_id):
    if not chat_id or not TELEGRAM_TOKEN:
        print("Telegram not configured: Missing token or chat ID")
        return
    try:
        bot = telebot.TeleBot(TELEGRAM_TOKEN)
        message = "عروض الطاقة الشمسية:\n"
        for offer in offers:
            message += f"{offer['title'][:40]} - {offer['price']} {offer['currency']} ({offer['governorate']})\n"
        bot.send_message(chat_id, message)
        print("Telegram message sent")
    except Exception as e:
        print(f"Error sending to Telegram: {e}")

# Enhanced GUI with more options
class SolarApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("نظام تتبع عروض الطاقة الشمسية في العراق")
        self.geometry("500x500")

        # Governorate filter
        self.gov_frame = ttk.LabelFrame(self, text="فلتر حسب المحافظة")
        self.gov_var = tk.StringVar(value="الكل")
        govs = ["الكل", "بغداد", "البصرة", "نينوى", "أربيل", "السليمانية", "الأنبار", "كربلاء"]
        for i, gov in enumerate(govs):
            ttk.Radiobutton(self.gov_frame, text=gov, variable=self.gov_var, value=gov).grid(row=0, column=i, padx=5)
        self.gov_frame.pack(pady=10, fill='x')

        # Source selection
        self.source_frame = ttk.LabelFrame(self, text="مصادر البيانات")
        self.google_var = tk.BooleanVar(value=True)
        self.opensooq_var = tk.BooleanVar(value=True)
        ttk.Checkbutton(self.source_frame, text="Google", variable=self.google_var).pack(side='left', padx=10)
        ttk.Checkbutton(self.source_frame, text="OpenSooq", variable=self.opensooq_var).pack(side='left', padx=10)
        self.source_frame.pack(pady=10, fill='x')

        # Output options
        self.output_frame = ttk.LabelFrame(self, text="خيارات الإخراج")
        self.pdf_var = tk.BooleanVar(value=True)
        self.excel_var = tk.BooleanVar()
        ttk.Checkbutton(self.output_frame, text="PDF", variable=self.pdf_var).pack(side='left', padx=10)
        ttk.Checkbutton(self.output_frame, text="Excel", variable=self.excel_var).pack(side='left', padx=10)
        self.output_frame.pack(pady=10, fill='x')

        # Telegram chat ID input
        self.telegram_frame = ttk.LabelFrame(self, text="إرسال إلى تيليغرام")
        self.chat_id_var = tk.StringVar()
        ttk.Entry(self.telegram_frame, textvariable=self.chat_id_var).pack(side='left', padx=5)
        ttk.Label(self.telegram_frame, text="أدخل Chat ID").pack(side='left', padx=5)
        self.telegram_frame.pack(pady=10, fill='x')

        # Search button
        ttk.Button(self, text="ابحث وعرض النتائج", command=self.run_search).pack(pady=10)

        # Query database button
        ttk.Button(self, text="استعلام قاعدة البيانات", command=self.query_db).pack(pady=10)

        # Status bar
        self.status = ttk.Label(self, text="جاهز", relief='sunken')
        self.status.pack(fill='x', side='bottom')

    def run_search(self):
        self.status.config(text="جارٍ البحث...")
        self.update()

        offers = []
        selected_gov = self.gov_var.get()

        # Get data from selected sources
        if self.google_var.get():
            google_offers = search_google()
            if selected_gov != "الكل":
                google_offers = [o for o in google_offers if fuzz.partial_ratio(selected_gov, o.get('governorate', '')) > 80]
            offers.extend(google_offers)

        if self.opensooq_var.get():
            opensooq_offers = scrape_opensooq()
            if selected_gov != "الكل":
                opensooq_offers = [o for o in opensooq_offers if fuzz.partial_ratio(selected_gov, o.get('governorate', '')) > 80]
            offers.extend(opensooq_offers)

        if not offers:
            self.status.config(text="لم يتم العثور على عروض")
            return

        # Store in database
        conn = sqlite3.connect('solar_offers.db')
        for offer in offers:
            category = categorize_offer(offer['title'], offer.get('snippet', ''))
            conn.execute("INSERT INTO offers (category, title, price, currency, governorate, source, link, date) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                        (category, offer['title'], offer['price'], offer['currency'], offer['governorate'], offer['source'], offer['link'], datetime.now().strftime("%Y-%m-%d")))
        conn.commit()
        conn.close()

        # Generate outputs
        if self.pdf_var.get():
            generate_pdf(offers)

        if self.excel_var.get():
            df = pd.DataFrame(offers)
            excel_name = f"solar_offers_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
            df.to_excel(excel_name, index=False)
            print(f"Excel file generated: {excel_name}")

        # Send to Telegram if chat ID provided
        if self.chat_id_var.get():
            send_to_telegram(offers, self.chat_id_var.get())

        self.status.config(text=f"تم العثور على {len(offers)} عرض")

    def query_db(self):
        conn = sqlite3.connect('solar_offers.db')
        query = "SELECT * FROM offers"
        params = []
        if self.gov_var.get() != "الكل":
            query += " WHERE governorate LIKE ?"
            params.append(f"%{self.gov_var.get()}%")
        df = pd.read_sql_query(query, conn, params=params)
        conn.close()

        if df.empty:
            self.status.config(text="لا توجد بيانات في قاعدة البيانات")
            return

        # Save query results to Excel for display
        excel_name = f"db_query_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        df.to_excel(excel_name, index=False)
        self.status.config(text=f"تم تصدير {len(df)} عرض إلى {excel_name}")

if __name__ == "__main__":
    setup_database()
    app = SolarApp()
    app.mainloop()

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


TclError: no display name and no $DISPLAY environment variable